In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
import yfinance as yf

In [29]:

df = pd.read_csv("analyst_ratings_processed.csv")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

df.rename(columns={
    "title": "headline",
    "stock": "ticker"
}, inplace=True)

In [30]:
top_tickers = (
    df['ticker']
    .value_counts()
    .head(1)
    .index
    .tolist()
)
print(len(top_tickers))


1


In [31]:
df_top = df[df['ticker'].isin(top_tickers)].copy()

In [32]:
import textblob

def get_sentiment(text):
    analysis=textblob.TextBlob(text).sentiment.polarity
    if analysis>0:
        return 1
    elif analysis<0:
        return -1
    else:
        return 0

df_top['sentiment']=df['headline'].apply(get_sentiment)
df_top

,headline,date,ticker,sentiment
838963,Shares of several healthcare companies are tra...,2020-06-11 10:22:00-04:00,MRK,0
838964,Johnson & Johnson To Start Coronavirus Vaccine...,2020-06-11 00:16:00-04:00,MRK,0
838965,The Daily Biotech Pulse: Keytruda Setback For ...,2020-06-10 07:30:00-04:00,MRK,0
838966,Merck Announces That The Phase 3 KEYNOTE-361 T...,2020-06-09 16:13:00-04:00,MRK,1
838967,"The Week Ahead In Biotech: Viela FDA Decision,...",2020-06-07 13:43:00-04:00,MRK,-1
...,...,...,...,...
842308,BenchmarkJournal.com Free Analyst Review for A...,2009-08-17 08:14:00-04:00,MRK,1
842309,Trends in the U.K. and Irish Pharmaceutical an...,2009-08-17 05:01:00-04:00,MRK,0
842310,ParagonReport.com Complimentary Market Update ...,2009-08-10 13:35:00-04:00,MRK,1
842311,ParagonReport.com Complimentary Market Update ...,2009-08-07 11:30:00-04:00,MRK,1


In [33]:
def majority_vote(s):
    return s.value_counts().idxmax()
sentiment_daily = (
    df_top.groupby(['ticker', 'date'])['sentiment']
          .apply(majority_vote)
          .reset_index()
)

In [34]:
sentiment_daily['date'] = pd.to_datetime(sentiment_daily['date'], utc=True)
sentiment_daily['date'] = sentiment_daily['date'].dt.tz_convert(None)


In [35]:
sentiment_continuous = []
for ticker, grp in sentiment_daily.groupby('ticker'):
    grp = grp.sort_values('date').set_index('date')
    all_days = pd.date_range(
        start=grp.index.min(),
        end=grp.index.max(),
        freq='B'
    )
    grp = grp.reindex(all_days)
    grp['sentiment'] = grp['sentiment'].fillna(0).astype(int)
    grp['ticker'] = ticker
    sentiment_continuous.append(grp.reset_index())
sentiment_continuous = pd.concat(sentiment_continuous, ignore_index=True)
sentiment_continuous.rename(columns={'index': 'date'}, inplace=True)

In [36]:
sentiment_continuous.set_index('date')

,ticker,sentiment
date,,
2009-07-27 12:33:00,MRK,-1
2009-07-28 12:33:00,MRK,0
2009-07-29 12:33:00,MRK,0
2009-07-30 12:33:00,MRK,0
2009-07-31 12:33:00,MRK,0
...,...,...
2020-06-05 12:33:00,MRK,0
2020-06-08 12:33:00,MRK,0
2020-06-09 12:33:00,MRK,0


In [37]:
import re
def normalize_ticker(t):
    t = str(t).upper().strip()
    t = re.sub(r'\s+(US|EQ|LN|JP|HK)$', '', t)
    t = re.sub(r'\.O$', '', t)
    t = re.sub(r'\.N$', '', t)
    if t.isalpha() and len(t) <= 10:
        return t

    return None 


In [38]:
df['ticker_clean'] = df['ticker'].apply(normalize_ticker)

df = df.dropna(subset=['ticker_clean'])
top_tickers = (
    df['ticker_clean']
    .value_counts()
    .head(1)
    .index
    .tolist()
)


In [39]:
price_data = []
start_date = pd.to_datetime(sentiment_continuous['date'].min()).tz_localize(None)
end_date   = pd.to_datetime(sentiment_continuous['date'].max()).tz_localize(None)
for ticker in top_tickers:
    prices = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        progress=False,
        threads=False
    )
    if prices is None or prices.empty:
        continue
    if isinstance(prices.columns, pd.MultiIndex):
        prices.columns = prices.columns.get_level_values(0)
    prices = prices[['Close']].reset_index()
    prices['ticker'] = ticker
    prices.columns = prices.columns.str.lower()
    price_data.append(prices)
print("Downloaded tickers:", len(price_data))
if not price_data:
    raise RuntimeError("No price data downloaded")
prices_df = pd.concat(price_data, ignore_index=True)


Downloaded tickers: 1


/var/folders/x9/0xsgqm2n559290vsdxtfl2dh0000gn/T/ipykernel_20183/1189389700.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(


In [40]:
print(prices_df.head())
print(prices_df.columns)


Price       date      close ticker
0     2009-07-27  16.760765    MRK
1     2009-07-28  16.352232    MRK
2     2009-07-29  16.270533    MRK
3     2009-07-30  16.308653    MRK
4     2009-07-31  16.346785    MRK
Index(['date', 'close', 'ticker'], dtype='object', name='Price')


In [41]:
def normalize_merge_keys(df):
    df = df.copy()
    df['ticker'] = (
        df['ticker']
        .astype(str)
        .str.strip()
        .str.upper()
    )
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].dt.tz_localize(None)
    df['date'] = df['date'].dt.normalize()

    return df


In [42]:
sentiment_continuous = normalize_merge_keys(sentiment_continuous)
prices_df = normalize_merge_keys(prices_df)


In [43]:
final_data = pd.merge(
    sentiment_continuous,
    prices_df,
    on=['date', 'ticker'],
    how='inner'
)

final_data = final_data.sort_values(['ticker', 'date']).reset_index(drop=True)


In [44]:
final_data.set_index('date')

,ticker,sentiment,close
date,,,
2009-07-27,MRK,-1,16.760765
2009-07-28,MRK,0,16.352232
2009-07-29,MRK,0,16.270533
2009-07-30,MRK,0,16.308653
2009-07-31,MRK,0,16.346785
...,...,...,...
2020-06-05,MRK,0,65.446350
2020-06-08,MRK,0,65.955528
2020-06-09,MRK,0,65.446350


In [45]:
final_data.to_csv('headlines_with_prices.csv')